# Learning music with Python (Potentially)

![SegmentLocal](img/music_eric.png "segment")

## Pygame guitar circles - Kirk Kaiser
https://www.makeartwithpython.com/blog/video-synthesizer-in-python/

![SegmentLocal](img/preview.gif "segment")


### Circles.py

In [ ]:
import pyaudio
import aubio
import numpy as np

import pygame
import random

from threading import Thread

import queue
import time

# Setup audio stream
p = pyaudio.PyAudio()

CHANNELS = 1
RATE = 44100
buffer_size = 4096  # needed to change this to get undistorted audio


# detect recorder
info = p.get_host_api_info_by_index(0)
numdevices = info.get("deviceCount")

recorder_name = "Scarlett 2i2 USB"
recorder_id = 1

for i in range(0, numdevices):
    name = p.get_device_info_by_host_api_device_index(0, i).get("name")
    if name == recorder_name:
        recorder_id = i

# Open audio stream
stream = p.open(
    format=pyaudio.paFloat32,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    input_device_index=recorder_id,
)

# setup onset detector
tolerance = 0.8
win_s = 4096  # fft size
hop_s = buffer_size // 2  # hop size
onset = aubio.onset("default", win_s, hop_s, RATE)

q = queue.Queue()

# Pygame
pygame.init()

# run in window
screenWidth, screenHeight = 800, 800
screen = pygame.display.set_mode((screenWidth, screenHeight))

clock = pygame.time.Clock()

white = (255, 255, 255)
black = (0, 0, 0)


class Circle(object):
    def __init__(self, x, y, color, size):
        self.x = x
        self.y = y
        self.color = color
        self.size = size

    def shrink(self):
        self.size -= 3


colors = [
    (229, 244, 227),
    (93, 169, 233),
    (0, 63, 145),
    (255, 255, 255),
    (109, 50, 109),
]
circleList = []


def draw_pygame():
    running = True
    while running:
        key = pygame.key.get_pressed()

        if key[pygame.K_q]:
            running = False
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        if not q.empty():
            b = q.get()
            newCircle = Circle(
                random.randint(0, screenWidth),
                random.randint(0, screenHeight),
                random.choice(colors),
                700,
            )
            circleList.append(newCircle)

        screen.fill(black)
        for place, circle in enumerate(circleList):
            if circle.size < 1:
                circleList.pop(place)
            else:
                pygame.draw.circle(
                    screen, circle.color, (circle.x, circle.y), circle.size
                )
            circle.shrink()

        pygame.display.flip()
        clock.tick(90)


def get_onsets():
    while True:
        try:
            buffer_size = 2048  # needed to change this to get undistorted audio
            audiobuffer = stream.read(buffer_size, exception_on_overflow=False)
            signal = np.fromstring(audiobuffer, dtype=np.float32)

            if onset(signal):
                q.put(True)

        except KeyboardInterrupt:
            print("*** Ctrl+C pressed, exiting")
            break


t = Thread(target=get_onsets, args=())
t.daemon = True
t.start()

draw_pygame()
get_onsets()

stream.stop_stream()
stream.close()
pygame.display.quit()


![SegmentLocal](img/circles_draw.png "segment")

## Audio to frec

![SegmentLocal](img/almost.jpeg "segment")

In [ ]:
%matplotlib inline
import pyaudio

import numpy as np
import matplotlib.pyplot as plt
from IPython import display
import time
import math

init the pyaudio object and detect my audio interface 

In [ ]:
p = pyaudio.PyAudio()

CHANNELS = 1
RATE = 44100
buffer_size = 44100


# detect recorder
info = p.get_host_api_info_by_index(0)
numdevices = info.get('deviceCount')
recorder_id = 1
recorder_name = 'Scarlett 2i2 USB'
for i in range(0, numdevices):
    name = p.get_device_info_by_host_api_device_index(0, i).get('name')
    # print(name)
    if name == recorder_name:
        recorder_id = i

# Open the stream
stream = p.open(
    format=pyaudio.paFloat32,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    input_device_index=recorder_id
)

lets plot some audio

In [ ]:
max(signal[-14685:])

In [ ]:
# make the plot as big as possible
plt.rcParams['figure.figsize'] = [16, 12]
plt.rcParams.update({'font.size': 18})

# constants
low_flag=True
dt = 1/buffer_size
t = np.arange(0,1,dt)

while True:
    audiobuffer = stream.read(buffer_size, exception_on_overflow=False)
    signal = np.frombuffer(audiobuffer, dtype=np.float32)
    # get the max volume from the last 1/3 of the signal
    last_3rd = int(buffer_size * (1/3))
    if max(signal[-last_3rd:]) > 0.03:
        low_flag=True
        note = signal
        plt.plot(t,signal,color='c')
        display.clear_output(wait=True)
        display.display(plt.show())
        time.sleep(0.2)
    else:
        if low_flag:
            print(f'low {max(signal)}')
            low_flag=False

lets explain the code

### Fast Fourier Transform

In [ ]:
Fs = RATE

n = len(note)     # Sample size
k = np.arange(n)  # range to sample
T = n/Fs          # Samples devided by RATE
plot_limit = 2000

frq = k/T # two sides frequency range
frq = frq[:len(frq)//2] # one side frequency range

Y = np.fft.fft(note)/n # dft and normalization
Y = Y[:n//2]

plt.plot(frq[:plot_limit],abs(Y[:plot_limit])) # plotting the spectrum
plt.xlabel('Freq (Hz)')
plt.ylabel('|Y(freq)|')
plt.show()

In [ ]:
index = np.argpartition(Y, -6)[-6:]
print(sorted(index))

lets detect what frecuecies we are seeing

In [ ]:
low_flag=True
min_volume_tolerance = 0.04
frec_volume_tolerance = 0.0008
all_frecs = []
while True:
    audiobuffer = stream.read(buffer_size, exception_on_overflow=False)
    signal = np.frombuffer(audiobuffer, dtype=np.float32)

    if max(signal) > min_volume_tolerance:
        low_flag=True
        ## FTT
        Fs = RATE

        n = len(signal)
        k = np.arange(n)
        T = n/Fs
        
        frq = k/T # two sides frequency range
        frq = frq[:len(frq)//2] # one side frequency range
        
        Y = np.fft.fft(signal)/n # dft and normalization
        Y = Y[:n//2]
        indices = Y > frec_volume_tolerance
        Y_clean = Y * indices

        if max(Y_clean) > 0:
            time.sleep(0.05)
            index = np.argpartition(Y, -4)[-4:]
            all_frecs.append(frq[index])
            print(index)
            print(min(frq[index]))
    else:
        if low_flag:
            print(f'low {max(signal)}')
            low_flag=False

## Infering notes base on frecuencies

In [ ]:
import math

def frequency_to_note(frequency):
    c4_frequency = 261.63

    notes = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]

    # Calculate the number of semitones away from A4
    semitones_away = 12 * (math.log2(frequency) - math.log2(c4_frequency))

    # Calculate the index of the closest note in the notes list
    print(semitones_away)
    closest_note_index = round(semitones_away) % 12

    # Get the closest note name
    closest_note = notes[closest_note_index]

    # Calculate the octave
    octave = 4 + math.floor(semitones_away / 12)

    # Return the note and octave as a string (e.g., "A4")
    return f"{closest_note}{octave}"

In [ ]:
frequency_to_note(262)

## Mathematically correct colored circles

In [ ]:
import pygame
import sys

import pyaudio

import numpy as np
import time
import math

from threading import Thread
import queue

import random

q = queue.Queue()

# audio init
p = pyaudio.PyAudio()
clock = pygame.time.Clock()

low_flag = True
min_volume_tolerance = 0.04
frec_volume_tolerance = 0.0008

CHANNELS = 1
RATE = 44100
buffer_size = 44100  # needed to change this to get undistorted audio


# detect recorder
info = p.get_host_api_info_by_index(0)
numdevices = info.get("deviceCount")
recorder_id = 1


recorder_name = "Scarlett 2i2 USB"
for i in range(0, numdevices):
    name = p.get_device_info_by_host_api_device_index(0, i).get("name")
    if name == recorder_name:
        recorder_id = i

stream = p.open(
    format=pyaudio.paFloat32,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    input_device_index=recorder_id,
)


# Initialize Pygame
pygame.init()

# Constants for the screen size
WIDTH, HEIGHT = 1280, 720
screenWidth, screenHeight = 1280, 720
WHITE = (255, 255, 255)
BROWN = (139, 69, 19)
BLACK = (0, 0, 0)

# Create the Pygame window
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Guitar Neck")

# Define the dimensions of the guitar neck
pygame.font.init()
font = pygame.font.SysFont("Arial", 30)

neck_width = 500
neck_length = WIDTH - WIDTH / 16
num_frets = 5
num_strings = 6  # Number of strings on the guitar

# fret offset
fret_offset = 10
# Calculate the width of each fret
fret_width = neck_length / num_frets - fret_offset

# Calculate the space between strings
string_spacing = neck_width / (num_strings - 1)

class Circle(object):
    def __init__(self, x, y, color, size):
        self.x = x
        self.y = y
        self.color = color
        self.size = size

    def shrink(self):
        self.size -= 3

circleList = []

def string_to_rgb(input_string):
    # Ensure the input string has at least two characters
    if len(input_string) < 2:
        raise ValueError("Input string must have at least two characters")

    # Extract the first and last characters
    first_char = input_string[0].upper()  # Convert to uppercase for case-insensitivity
    last_char = input_string[-1]

    # Map the first character to an RGB value
    first_char_mapping = ord(first_char) - ord('A')
    r = int((first_char_mapping / 6) * 255)

    # Map the last character to an RGB value
    last_char_mapping = int(last_char) * 25
    g = b = min(last_char_mapping, 255)

    return r, g, b


# Main loop
def game_lop():
    note = ["", 0]
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                # Quit Pygame
                pygame.quit()
                sys.exit()

        # Clear the screen
        screen.fill(BLACK)

        if not q.empty():
            note = q.get()
            print(note)
            newCircle = Circle(
                random.randint(0, screenWidth),
                random.randint(0, screenHeight),
                string_to_rgb(note[0]),
                700,
            )
            circleList.append(newCircle)

        # Draw circle
        for place, circle in enumerate(circleList):
            if circle.size < 1:
                circleList.pop(place)
            else:
                pygame.draw.circle(
                    screen, circle.color, (circle.x, circle.y), circle.size
                )
            circle.shrink()

        # draw text
        text = font.render(note[0], True, WHITE)
        screen.blit(text, (0, 0))

        
        pygame.display.flip()

        # Update the display
        pygame.display.flip()
        clock.tick(90)


def frequency_to_note(frequency):
    a4_frequency = 261.63  # C4
    notes = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]

    semitones_away = 12 * (math.log2(frequency) - math.log2(a4_frequency))
    closest_note_index = round(semitones_away) % 12
    closest_note = notes[closest_note_index]
    octave = 4 + math.floor(semitones_away / 12)

    return f"{closest_note}{octave}"


def audio_listener():
    while True:
        audiobuffer = stream.read(buffer_size, exception_on_overflow=False)
        signal = np.frombuffer(audiobuffer, dtype=np.float32)

        if max(signal) > min_volume_tolerance:
            ## FTT
            Fs = RATE

            n = len(signal)
            k = np.arange(n)
            T = n / Fs

            frq = k / T  # two sides frequency range
            frq = frq[: len(frq) // 2]  # one side frequency range

            Y = np.fft.fft(signal) / n  # dft and normalization
            Y = Y[: n // 2]
            indices = Y > frec_volume_tolerance
            Y_clean = Y * indices

            if max(Y_clean) > 0:
                time.sleep(0.05)
                index = np.argpartition(Y, -4)[-4:]
                frec = min(frq[index])
                q.put([frequency_to_note(frec), frec])


t = Thread(target=audio_listener, args=())
t.daemon = True
t.start()

game_lop()
audio_listener()

stream.stop_stream()
stream.close()
pygame.display.quit()


## Understanding the guitar fret

![SegmentLocal](img/fret.png "segment")

In [ ]:
import pygame
import sys

import pyaudio

import numpy as np
import time
import math

from threading import Thread
import queue

import random

q = queue.Queue()

# audio init
p = pyaudio.PyAudio()
clock = pygame.time.Clock()

low_flag = True
min_volume_tolerance = 0.04
frec_volume_tolerance = 0.0008

CHANNELS = 1
RATE = 44100
buffer_size = 44100  # needed to change this to get undistorted audio


# detect recorder
info = p.get_host_api_info_by_index(0)
numdevices = info.get("deviceCount")
recorder_id = 1


recorder_name = "Scarlett 2i2 USB"
for i in range(0, numdevices):
    name = p.get_device_info_by_host_api_device_index(0, i).get("name")
    if name == recorder_name:
        recorder_id = i

stream = p.open(
    format=pyaudio.paFloat32,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    input_device_index=recorder_id,
)


# Initialize Pygame
pygame.init()

# Constants for the screen size
WIDTH, HEIGHT = 1280, 720
screenWidth, screenHeight = 1280, 720
WHITE = (255, 255, 255)
RED = (225, 20, 20)
GREY = (200, 200, 200)
BROWN = (139, 69, 19)
BLACK = (0, 0, 0)

# Create the Pygame window
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Guitar Neck")

# Define the dimensions of the guitar neck
pygame.font.init()
font = pygame.font.SysFont("Arial", 30)

neck_width = 500
neck_length = WIDTH - WIDTH / 16
num_frets = 5
num_strings = 6  # Number of strings on the guitar

# fret offset
fret_offset = 10
# Calculate the width of each fret
fret_width = neck_length / num_frets - fret_offset

# Calculate the space between strings
string_spacing = neck_width / (num_strings - 1)


class Circle(object):
    def __init__(self, x, y, color, size):
        self.x = x
        self.y = y
        self.color = color
        self.size = size

    def shrink(self):
        self.size -= 3


circleList = []


# Main loop
def game_lop():
    note = ["", 0]
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                # Quit Pygame
                pygame.quit()
                sys.exit()

        # Clear the screen
        screen.fill(BLACK)

        if not q.empty():
            note = q.get()

        # draw text
        text = font.render(note[0], True, WHITE)
        screen.blit(text, (0, 0))

        e4_string = [
            "e4",
            "f4",
            "f#4",
            "g4",
            "g#4",
            "a4",
            "a#4",
            "b4",
            "c5",
            "c#5",
            "d5",
            "d#5",
            "e5",
            "f5",
        ]
        b3_string = [
            "b3",
            "c4",
            "c#4",
            "d4",
            "d#4",
            "e4",
            "f4",
            "f#4",
            "g4",
            "g#4",
            "a4",
            "a#4",
            "b4",
            "c5",
        ]
        g3_string = [
            "g3",
            "g#3",
            "a3",
            "a#3",
            "b3",
            "c4",
            "c#4",
            "d4",
            "d#4",
            "e4",
            "f4",
            "f#4",
            "g4",
            "g#4",
        ]
        d3_string = [
            "d3",
            "d#3",
            "e3",
            "f3",
            "f#3",
            "g3",
            "g#3",
            "a3",
            "a#3",
            "b3",
            "c4",
            "c#4",
            "d4",
            "d#4",
        ]
        a2_string = [
            "a2",
            "a#2",
            "b2",
            "c3",
            "c#3",
            "d3",
            "d#3",
            "e3",
            "f3",
            "f#3",
            "g3",
            "g#3",
            "a3",
            "a#3",
        ]
        e2_string = [
            "e2",
            "f2",
            "f#2",
            "g2",
            "g#2",
            "a2",
            "a#2",
            "b2",
            "c3",
            "c#3",
            "d3",
            "d#3",
            "e3",
            "f3",
        ]

        num_string = [f"{x}" for x in range(0, 14)]

        strings = [
            e4_string,
            b3_string,
            g3_string,
            d3_string,
            a2_string,
            e2_string,
            num_string,
        ]

        for i, string in enumerate(strings):
            for j, string_note in enumerate(string):
                note_color = WHITE
                if string_note.lower() == note[0].lower():
                    note_color = RED
                    # draw a square around the note
                    pygame.draw.rect(
                        screen,
                        GREY,
                        (80 + j * 80, 40 + i * 80, 80, 80),
                        0,
                    )
                text = font.render(string_note, True, note_color)
                screen.blit(text, (80 + j * 80, 40 + i * 80))
                line_lenght = 2
                if j == 0:
                    line_lenght = 5
                # draw a line to separate note
                if i < len(strings) - 1:
                    pygame.draw.line(
                        screen,
                        WHITE,
                        (160 + j * 80, 40 + i * 80),
                        (160 + j * 80, 40 + i * 80 + 80),
                        line_lenght,
                    )
            # draw a line to separate strings
            pygame.draw.line(
                screen,
                WHITE,
                (80, 40 + i * 80),
                (80 + len(string) * 80, 40 + i * 80),
                2,
            )

        pygame.display.flip()

        # Update the display
        pygame.display.flip()
        clock.tick(90)


def frequency_to_note(frequency):
    a4_frequency = 261.63  # C4
    notes = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]

    semitones_away = 12 * (math.log2(frequency) - math.log2(a4_frequency))
    closest_note_index = round(semitones_away) % 12
    closest_note = notes[closest_note_index]
    octave = 4 + math.floor(semitones_away / 12)

    return f"{closest_note}{octave}"


def audio_listener():
    while True:
        audiobuffer = stream.read(buffer_size, exception_on_overflow=False)
        signal = np.frombuffer(audiobuffer, dtype=np.float32)

        if max(signal) > min_volume_tolerance:
            ## FTT
            Fs = RATE

            n = len(signal)
            k = np.arange(n)
            T = n / Fs

            frq = k / T  # two sides frequency range
            frq = frq[: len(frq) // 2]  # one side frequency range

            Y = np.fft.fft(signal) / n  # dft and normalization
            Y = Y[: n // 2]
            indices = Y > frec_volume_tolerance
            Y_clean = Y * indices

            last_3rd = int(buffer_size * (1 / 3))
            if max(signal[-last_3rd:]) > 0.03:
                if max(Y_clean) > 0:
                    # time.sleep(0.005)
                    index = np.argpartition(Y, -4)[-4:]
                    frec = min(frq[index])
                    q.put([frequency_to_note(frec), frec])
            else:
                q.put(["", 0])


t = Thread(target=audio_listener, args=())
t.daemon = True
t.start()

game_lop()
audio_listener()

stream.stop_stream()
stream.close()
pygame.display.quit()
